# 15.0 简介

核心知识点：
- KNN（K-Nearest Neighbors，K 近邻）是经典的有监督学习算法，可用于分类和回归，属于懒惰学习器（Lazy Learner），无显式训练过程，仅在预测时计算距离。
- 核心原理：计算未知样本与训练集中所有样本的距离，将其归为距离最近的k个样本中占比最高的类别（分类），或取k个样本的均值 / 中位数（回归）。
- 算法性能高度依赖特征尺度和k 值选择，是本章核心调优方向，本章重点讲解基于 scikit-learn 的 KNN 模型创建、调整与评估。

# 15.1 找到一个观察值的最近邻

In [1]:
# 加载库
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from notebooks.Chapter12_Model_Selection.ch12_Model_selection import search_space

# 加载数据
iris = datasets.load_iris()
features= iris.data

# 创建标准化器
standardizer = StandardScaler()

# 特征标准化
features_standardized = standardizer.fit_transform(features)

# 两个最近的观察值
nearest_neighbors = NearestNeighbors(n_neighbors=2).fit(features_standardized)

# 创建一个观察值
new_observation = [1,1,1,1]

# 获取离观察值最近的两个观察值的索引，以及到这两个点的距离
distances ,indices = nearest_neighbors.kneighbors([new_observation])

# 查看最近的两个观察值
features_standardized[indices]

array([[[1.03800476, 0.55861082, 1.10378283, 1.18556721],
        [0.79566902, 0.32841405, 0.76275827, 1.05393502]]])

可以用metric参数设定距离指标：

In [3]:
# 找到按照欧氏距离来算最近的两个邻居
nearestneighbors_euclidean = NearestNeighbors(n_neighbors=2,metric='euclidean').fit(features_standardized)

还可以创建矩阵来表示离每个观察值最近的邻居：

In [6]:
# 寻找每个观察值按照欧氏距离极端的最近的3个邻居（包括他自己）
nearest_neighbors_euclidean = NearestNeighbors(n_neighbors=3,metric = 'euclidean').fit(features_standardized)

# 每个观察值和他最近的三个邻居的列表（包括他自己）
nearest_neighbors_with_self = nearestneighbors_euclidean.kneighbors_graph(features_standardized).toarray()

# 从最近的邻居中移除自己
for i,x in enumerate(nearest_neighbors_with_self):
    x[i]=0

# 查看离第一个观察值最近的两个邻居
nearest_neighbors_with_self[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## 核心知识点

### 1. 特征标准化是必做步骤
距离计算对特征单位敏感（如厘米和米的数值差异），需用`StandardScaler`将特征标准化为均值 0、方差 1，保证各特征对距离的贡献均等。

---

### 2. 常用距离度量指标（通过 `metric` 参数指定）
- **欧氏距离（默认推荐）**：
$$
d_{euclidean} = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}
$$
- **曼哈顿距离**：
$$
d_{manhattan} = \sum_{i=1}^n |x_i - y_i|
$$
- **闵可夫斯基距离**：
$$
d_{minkowski} = \left(\sum_{i=1}^n |x_i - y_i|^p\right)^{1/p}
$$
（$p=2$ 为欧氏距离，$p=1$ 为曼哈顿距离）

---

### 3. 核心方法
- `kneighbors()`：输入新样本，返回其最近 k 个邻居的距离数组和索引数组。
- `kneighbors_graph()`：生成样本的邻居关系矩阵，矩阵值为 0/1，表示样本间是否为最近邻，用于分析样本相似性。

# 15.2 创建一个KNN分类器

In [12]:
# 加载库
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

# 加载数据
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 创建standardizer
standardizer = StandardScaler()

# 标准化特征
X_std = standardizer.fit_transform(X)

# 训练一个有五个邻居的KNN分类器
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1).fit(X_std, y)

# 创建两个观察值
new_observations = [[0.75,0.75,0.75,0.75],[1,1,1,1]]

# 观测这两个观察值的分类
knn.predict(new_observation)

array([1, 2])

In [13]:
# 查看每个观察值分别属于三个分类中的某一个的概率
knn.predict_proba(new_observations)

array([[0. , 0.6, 0.4],
       [0. , 0. , 1. ]])

In [14]:
knn.predict(new_observations)

array([1, 2])

## 一、问题描述

对于分类未知的观察值，基于邻居的分类来预测它的分类。

---

## 二、KNN算法原理

### 2.1 工作机制
1. **找邻居**：基于距离指标（如欧氏距离）找到最近的 $k$ 个观察值（称为 $x$ 的**邻域**）
2. **投票**：这 $k$ 个观察值基于自己的分类来"投票"
3. **决策**：得票最多的分类即为预测分类

### 2.2 分类概率公式

属于某个分类 $j$ 的概率：

$$P(y=j) = \frac{1}{k}\sum_{i \in v} I(y_i = j)$$

其中：
- $v$：$x$ 的邻域内的 $k$ 个观察值
- $y_i$：第 $i$ 个观察值的分类
- $I$：指示函数（函数值为1表示真，为0表示假）

---

## 三、关键参数

| 参数 | 说明 |
|:---|:---|
| `n_neighbors` | 邻居数量 $k$ |
| `metric` | 距离指标（详见14.1节） |
| `n_jobs` | CPU内核数，`-1`表示使用全部核心 |
| `algorithm` | 计算最近邻居的算法（自动选择最优） |
| `weights` | 投票权重，`distance`表示距离越近权重越高 |

---

## 四、⚠️ 重要注意事项

&gt; **标准化特征至关重要！**
&gt;
&gt; 因为计算距离时所有的特征被认为是在**同一单位**下的，所以在使用KNN分类器之前使用 `StandardScaler` 标准化特征是很重要的。

---

## 五、预测结果解读

| 观察值 | 预测分类 | 各类概率 | 说明 |
|:---|:---:|:---|:---|
| 第1个 | 1 | $[0.0,\ 0.6,\ 0.4]$ | 分类1概率最高(0.6)，但不绝对 |
| 第2个 | 2 | $[0.0,\ 0.0,\ 1.0]$ | 分类2概率为1.0，确定性最高 |

# 15.3 确定最佳的邻域点集的大小

In [15]:
# 加载库
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import GridSearchCV

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 创建standardizer
standardizer = StandardScaler()

# 标准化特征
features_standardized = standardizer.fit_transform(features)

# 创建一个KNN分类器
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)

# 创建一个流水线
pipe = Pipeline([('standardizer',standardizer),('knn',knn)])

# 确定一个可选值的范围
search_space = [{'knn__n_neighbors':[1,2,3,4,5,6,7,8,9,10]}]

# 创建grid搜索
classifier = GridSearchCV(pipe,search_space,cv= 5,verbose=0).fit(features_standardized,target)

In [16]:
# 最佳邻域值的大小
classifier.best_estimator_.get_params()['knn__n_neighbors']

6

## 核心知识点

1. **k值的偏差 - 方差权衡**: KNN模型的泛化能力由k值决定，是调优核心：
   - k太小：模型复杂，仅关注局部样本，**偏差小、方差大**，易过拟合（对噪声样本敏感）。
   - k太大：模型简单，关注全局样本，**偏差大、方差小**，易欠拟合（无法捕捉数据局部特征）。
   - 最优k：找到偏差和方差的平衡点，使模型泛化能力最强。

2. **网格搜索 + 交叉验证**: 是寻找最优k的标准方法，通过GridSearchCV遍历指定的k值搜索空间，结合交叉验证评估模型性能，选择得分最高的k。

3. **Pipeline流水线的必要性**: 将标准化和KNN整合为一个流程，避免交叉验证时训练集的标准化参数泄露到测试集，保证模型评估的公平性和可靠性。

# 15.4 创建一个基于半径的最近邻分类器

In [17]:
# 加载库
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 创建Standardizer
standardizer = StandardScaler()

# 标准化特征
features_standardized = standardizer.fit_transform(features)

# 训练一个基于半径的最近邻分类器
rnn = RadiusNeighborsClassifier(radius = .5,n_jobs=-1).fit(features_standardized,target)

# 创建两个观测值
new_observation = [[1,1,1,1]]

# 预测这两个观测值的分类
rnn.predict(new_observation)

array([2])

## 核心知识点

1. **基于半径的近邻（RNN）**：是 KNN 的变体，核心区别在于**邻居选择规则**：
   - KNN：固定邻居数量 \( k \)，找距离最近的 \( k \) 个样本。
   - RNN：固定距离范围 \( radius \)，找该半径内的**所有样本**作为邻居。

2. **核心参数**：
   - \( radius \)：指定邻居的距离范围，需通过网格搜索确定最优值（过小易无邻居，过大易欠拟合）。
   - \( outlier\_label \)：可选参数，指定当样本半径内无邻居时的预测标签，可用于**异常点检测**。
   - \( n\_jobs \)、\( weights \)、\( n\_jobs \) 参数与 KNN 一致。

3. **适用场景**：适合**样本密度不均匀**的数据集（如部分区域样本密集、部分区域稀疏），或需要根据**局部密度进行**分类的任务；样本密度均匀时，KNN 性能更稳定。

4. **局限性**：对 \( radius \) 值高度敏感，若半径设置不当，易出现"无邻居"或"邻居过多"的情况，实际使用中不如 KNN 广泛。